In [1]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [2]:
!pip install gdown

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
!gdown --id 1-Y70PaxrtT9suVN3a9jZkvU932VPUR4K

/usr/local/lib/python3.8/site-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-Y70PaxrtT9suVN3a9jZkvU932VPUR4K
To: /kaggle/working/sequences.txt
100%|███████████████████████████████████████| 1.58M/1.58M [00:00<00:00, 143MB/s]


In [4]:
!gdown --id 1-Pm4dOGP8n8eXx458mNSOKC4dLS5aQc8

/usr/local/lib/python3.8/site-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-Pm4dOGP8n8eXx458mNSOKC4dLS5aQc8
To: /kaggle/working/graph_labels.txt
100%|██████████████████████████████████████| 42.9k/42.9k [00:00<00:00, 50.9MB/s]


In [5]:
#Prot_model
!gdown --id 1MHWfO0Ym1wuagOckzUcL2aDMYiwr8f3c

/usr/local/lib/python3.8/site-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MHWfO0Ym1wuagOckzUcL2aDMYiwr8f3c
To: /kaggle/working/model_ProBert_3.pth
100%|███████████████████████████████████████| 1.68G/1.68G [00:13<00:00, 122MB/s]


In [6]:
import os 
os.chdir("/kaggle/working/")

In [7]:
#!git clone https://github.com/mani-aiml/amazon-sagemaker-protein-classification.git

In [8]:
import torch
torch.cuda.is_available()

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


False

In [9]:
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset

class ProteinSequenceDataset(Dataset):
    def __init__(self, sequence, targets, tokenizer, max_len):
        self.sequence = sequence
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, item):
        sequence = str(self.sequence[item])
        target = self.targets[item]
        encoding = self.tokenizer.encode_plus(
            sequence,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
          'protein_sequence': sequence,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

In [10]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 42.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 79.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.3/772.3 KB 43.6 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import torch.nn.functional as F
import torch.nn as nn


PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert_bfd_localization'
class ProteinClassifier(nn.Module):
    def __init__(self, n_classes):
        super(ProteinClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.classifier = nn.Sequential(nn.Dropout(p=0.2),
                                        nn.Linear(self.bert.config.hidden_size, n_classes),
                                        nn.Tanh())
        
    def forward(self, input_ids, attention_mask):
        output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        return self.classifier(output.pooler_output)

2023-01-19 00:06:45.285355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 00:06:55.815660: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-19 00:06:55.815796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-19 00:06:55.815808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [12]:
pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 2.8 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 KB 21.2 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [14]:
%tb
# from __future__ import print_function

import argparse
# import json
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import Dataset, DataLoader, RandomSampler, TensorDataset
from transformers import BertTokenizer, get_linear_schedule_with_warmup
import torch_optimizer as optim

# Network definition
# from model_def import ProteinClassifier
# from data_prep import ProteinSequenceDataset
 
## SageMaker Distributed code.
# from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
# import smdistributed.dataparallel.torch.distributed as dist

# dist.init_process_group()

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 512  # this is the max length of the sequence
PRE_TRAINED_MODEL_NAME = 'Rostlab/prot_bert'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME, do_lower_case=False)

training_dir = "/kaggle/working/"

sequences = list()
print(training_dir+'sequences.txt')
with open(training_dir+'sequences.txt', 'r') as f:
    for line in f:
        sequences.append(line[:-1])

# Split data into training and test sets
sequences_train = list()
sequences_test = list()
proteins_test = list()
y_train = list()
with open(training_dir+'graph_labels.txt', 'r') as f:
    for i,line in enumerate(f):
        t = line.split(',')
        if len(t[1][:-1]) == 0:
            proteins_test.append(t[0])
            sequences_test.append(sequences[i])
        else:
            sequences_train.append(sequences[i])
            y_train.append(int(t[1][:-1]))
from sklearn.model_selection import train_test_split

sequences_train_ , sequences_val, y_train_, y_val = train_test_split(sequences_train,y_train ,
                                   random_state=104, 
                                   test_size=0.1, 
                                   shuffle=True)
  
def _get_train_data_loader(batch_size, training_dir):

    # Read sequences
#     sequences = list()
#     print(training_dir+'sequences.txt')
#     with open(training_dir+'sequences.txt', 'r') as f:
#         for line in f:
#             sequences.append(line[:-1])

#     # Split data into training and test sets
#     sequences_train = list()
#     sequences_test = list()
#     proteins_test = list()
#     y_train = list()
#     with open(training_dir+'graph_labels.txt', 'r') as f:
#         for i,line in enumerate(f):
#             t = line.split(',')
#             if len(t[1][:-1]) == 0:
#                 proteins_test.append(t[0])
#                 sequences_test.append(sequences[i])
#             else:
#                 sequences_train.append(sequences[i])
#                 y_train.append(int(t[1][:-1]))
    sequences_train = sequences_train_
    y_train =y_train_
    sequences_train = np.array(sequences_train)
    y_train = np.array(y_train)

    train_data = ProteinSequenceDataset(
        sequence=sequences_train,
        targets=y_train,
        tokenizer=tokenizer,
        max_len=MAX_LEN
  )
    # train_sampler = torch.utils.data.distributed.DistributedSampler(
    #         dataset,
    #         num_replicas=dist.get_world_size(),
    #         rank=dist.get_rank())
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    return train_dataloader

def _get_test_data_loader(batch_size, training_dir):
        # Read sequences
#     sequences = list()
#     with open(training_dir+'sequences.txt', 'r') as f:
#         for line in f:
#             sequences.append(line[:-1])

    # Split data into training and test sets
#     sequences_train = list()
#     sequences_test = list()
#     proteins_test = list()
#     y_train = list()
#     with open(training_dir+'graph_labels.txt', 'r') as f:
#         for i,line in enumerate(f):
#             t = line.split(',')
#             if len(t[1][:-1]) == 0:
#                 proteins_test.append(t[0])
#                 sequences_test.append(sequences[i])
#             else:
#                 sequences_train.append(sequences[i])
#                 y_train.append(int(t[1][:-1]))
    sequences_test = sequences_val
    proteins_test = y_val
    sequences_test = np.array(sequences_test)
    proteins_test = np.array(proteins_test)

    test_data = ProteinSequenceDataset(
        sequence=sequences_test,
        targets=proteins_test,
        tokenizer=tokenizer,
        max_len=MAX_LEN
  )
    # test_sampler = RandomSampler(test_data)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    return test_dataloader

def freeze(model, frozen_layers):
    modules = [model.bert.encoder.layer[:frozen_layers]] 
    for module in modules:
        for param in module.parameters():
            param.requires_grad = False
            
def train(batch_size,data_dir,test_batch_size,test_dir,frozen_layers,num_labels, lr, epsilon, weight_decay, epochs, log_interval, verbose, model_dir):
    #use_cuda = args.num_gpus > 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # world_size = dist.get_world_size()
    # rank = dist.get_rank()
    # local_rank = dist.get_local_rank()
    
    # set the seed for generating random numbers
    # torch.manual_seed(args.seed)
    # if use_cuda:
    #     torch.cuda.manual_seed(args.seed)

    train_loader = _get_train_data_loader(batch_size, data_dir)
    #if rank == 0:
    test_loader = _get_test_data_loader(test_batch_size, test_dir)
    print("Max length of sequence: ", MAX_LEN)
    print("Freezing {} layers".format(frozen_layers))
    print("Model used: ", PRE_TRAINED_MODEL_NAME)

    # logger.debug(
    #     "Processes {}/{} ({:.0f}%) of train data".format(
    #         len(train_loader.sampler),
    #         len(train_loader.dataset),
    #         100.0 * len(train_loader.sampler) / len(train_loader.dataset),
    #     ))

    model = ProteinClassifier(
        num_labels  # The number of output labels.
    )
    freeze(model, frozen_layers)
    model = model.to(device) #DDP(model.to(device), broadcast_buffers=False)
    # torch.cuda.set_device('local_rank')
    # model.cuda(local_rank)
    continue_training = True
    
    if continue_training :
        print("=====PRETRAINED MODEL======")
        #load checkpoint
        checkpoint = torch.load("/kaggle/working/model_ProBert_3.pth")
        #load state dict
        model.load_state_dict(checkpoint)
    
    else:
        print("=====Fine Tuning FROM ProBert======")
    optimizer = optim.Lamb(
            model.parameters(), 
            lr = lr ,  #* dist.get_world_size()
            betas=(0.9, 0.999), 
            eps=epsilon, 
            weight_decay=weight_decay)
    total_steps = len(train_loader.dataset)
    
    scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=total_steps)
    
    loss_fn = nn.CrossEntropyLoss().to(device)
        
    for epoch in range(1, epochs + 1):
        model.train()
        for step, batch in enumerate(train_loader):
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_labels = batch['targets'].to(device)

            outputs = model(b_input_ids,attention_mask=b_input_mask)
            loss = loss_fn(outputs, b_labels)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()
            optimizer.zero_grad()
            
            if step % log_interval == 0 : #and rank == 0
                logger.info(
                    "Collecting data from Master Node: \n Train Epoch: {} [{}/{} ({:.0f}%)] Training Loss: {:.6f}".format(
                        epoch,
                        step * len(batch['input_ids']),  #*world_size
                        len(train_loader.dataset),
                        100.0 * step / len(train_loader),
                        loss.item(),
                    ))
            if verbose:
                print('Batch', step)
        test(model, test_loader, device)
        scheduler.step()
        model_save = model.module if hasattr(model, "module") else model
        save_model(model_save, model_dir, epoch)

def save_model(model, model_dir, i):
    path = os.path.join(model_dir, f'model_ProBert_{i+3}.pth')
    # recommended way from http://pytorch.org/docs/master/notes/serialization.html
    torch.save(model.state_dict(), path)
    logger.info(f"Saving model: {path} \n")


def test(model, test_loader, device):
    model.eval()
    losses = []
    correct_predictions = 0
    loss_fn = nn.CrossEntropyLoss().to(device)
    tmp_eval_accuracy, eval_accuracy = 0, 0
    
    with torch.no_grad():
        for batch in test_loader:
            b_input_ids = batch['input_ids'].to(device)
            b_input_mask = batch['attention_mask'].to(device)
            b_labels = batch['targets'].to(device)

            outputs = model(b_input_ids,attention_mask=b_input_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, b_labels)
            correct_predictions += torch.sum(preds == b_labels)
            losses.append(loss.item())
            
    print('\nTest set: Validation loss: {:.4f}, Validation Accuracy: {:.0f}%\n'.format(
        np.mean(losses),
        100. * correct_predictions.double() / len(test_loader.dataset)))

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()

#     # Data and model checkpoints directories
#     parser.add_argument("--num_labels", type=int, default=18, metavar="N", help="input batch size for training (default: 10)")

#     parser.add_argument("--batch-size", type=int, default=8, metavar="N", help="input batch size for training (default: 1)")
#     parser.add_argument("--test-batch-size", type=int, default=8, metavar="N", help="input batch size for testing (default: 8)")
#     parser.add_argument("--epochs", type=int, default=10, metavar="N", help="number of epochs to train (default: 2)")
#     parser.add_argument("--lr", type=float, default=0.3e-5, metavar="LR", help="learning rate (default: 0.3e-5)")
#     parser.add_argument("--weight_decay", type=float, default=0.01, metavar="M", help="weight_decay (default: 0.01)")
#     parser.add_argument("--seed", type=int, default=43, metavar="S", help="random seed (default: 43)")
#     parser.add_argument("--epsilon", type=int, default=1e-8, metavar="EP", help="random seed (default: 1e-8)")
#     parser.add_argument("--frozen_layers", type=int, default=10, metavar="NL", help="number of frozen layers(default: 10)")
#     parser.add_argument('--verbose', action='store_true', default=False,help='For displaying SMDataParallel-specific logs')
#     parser.add_argument("--log-interval",type=int,default=10,metavar="N",help="how many batches to wait before logging training status",)
   
#     # Container environment
#     # parser.add_argument("--hosts", type=list, default=json.loads(os.environ["SM_HOSTS"]))
#     # parser.add_argument("--current-host", type=str, default=os.environ["SM_CURRENT_HOST"])
#     parser.add_argument("--model-dir", type=str, default="/Models")
#     parser.add_argument("--data-dir", type=str, default="/content/drive/MyDrive/Challenge")
#     parser.add_argument("--test", type=str, default="/content/drive/MyDrive/Challenge")
#     # parser.add_argument("--num-gpus", type=int, default=os.environ["SM_NUM_GPUS"])

#     train(parser.parse_args())

batch_size = 8
data_dir = "/kaggle/working/"
test_batch_size = 8
test_dir = "/kaggle/working/"
frozen_layers = 15
num_labels = 18
lr = 0.3e-5
epsilon = 1e-8
weight_decay = 0.01
epochs = 10
log_interval = 10
verbose = False
model_dir ="/kaggle/working/"
train(batch_size,data_dir,test_batch_size,test_dir,frozen_layers,num_labels, lr, epsilon, weight_decay, epochs, log_interval, verbose, model_dir)

No traceback available to show.
Downloading: 100%|██████████| 81.0/81.0 [00:00<00:00, 76.7kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 92.4kB/s]
Downloading: 100%|██████████| 86.0/86.0 [00:00<00:00, 68.7kB/s]
Downloading: 100%|██████████| 361/361 [00:00<00:00, 421kB/s]


/kaggle/working/sequences.txt
Max length of sequence:  512
Freezing 15 layers
Model used:  Rostlab/prot_bert


Downloading: 100%|██████████| 1.68G/1.68G [00:53<00:00, 31.5MB/s] 
Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


=====PRETRAINED MODEL======
 Train Epoch: 1 [0/4399 (0%)] Training Loss: 2.785866
 Train Epoch: 1 [80/4399 (2%)] Training Loss: 2.811027
 Train Epoch: 1 [160/4399 (4%)] Training Loss: 2.784500
 Train Epoch: 1 [240/4399 (5%)] Training Loss: 2.664945
 Train Epoch: 1 [320/4399 (7%)] Training Loss: 2.850210
 Train Epoch: 1 [400/4399 (9%)] Training Loss: 2.713369
 Train Epoch: 1 [480/4399 (11%)] Training Loss: 2.791614
 Train Epoch: 1 [560/4399 (13%)] Training Loss: 2.841779
 Train Epoch: 1 [640/4399 (15%)] Training Loss: 2.779568
 Train Epoch: 1 [720/4399 (16%)] Training Loss: 2.841230
 Train Epoch: 1 [800/4399 (18%)] Training Loss: 2.799253
 Train Epoch: 1 [880/4399 (20%)] Training Loss: 2.798998
 Train Epoch: 1 [960/4399 (22%)] Training Loss: 2.672273
 Train Epoch: 1 [1040/4399 (24%)] Training Loss: 2.772959
 Train Epoch: 1 [1120/4399 (25%)] Training Loss: 2.695515
 Train Epoch: 1 [1200/4399 (27%)] Training Loss: 2.826844
 Train Epoch: 1 [1280/4399 (29%)] Training Loss: 2.777010
 Train E

KeyboardInterrupt: 